<a href="https://colab.research.google.com/github/AlexH20/Supervised-ML-sentiment-measures/blob/main/Merge_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas_market_calendars
!pip install holidays

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 89 kB 3.5 MB/s 
     |████████████████████████████████| 152 kB 35.4 MB/s 
  Created wheel for exchange-calendars: filename=exchange_calendars-3.6.3-py3-none-any.whl size=182636 sha256=096ade0e6370fe6f4a35096c43c88fc6532e62918f14a513efae8db16cc58301
  Stored in directory: /root/.cache/pip/wheels/b8/a3/19/b4611514d34ffd61d13aef10fefc2dcaf3754145121ceba647
Successfully built exchange-calendars


In [ ]:
import pandas as pd
import csv
from datetime import datetime, timedelta, time
from google.colab import drive 
import sys
import re
import math
import holidays
import pandas_market_calendars as mcal

drive.mount("/content/gdrive")

#Functions to concatenate text

def concatenate_text(g):
    return ' '.join(g.Text_unprocessed)

#Function to count words of text using regex

def count_words(Text):
    word_count = len(re.findall(r'\w+', Text))
    return word_count

# Function to check whether date is on a non-trading day or news article is published later than 4pm. If so, then return next trading day.

holidaysUS = holidays.US()
nyse = mcal.get_calendar('NYSE')
stock_holidays = nyse.holidays()

stock_holidays = list(pd.to_datetime(stock_holidays.holidays))
stock_holidays = [x.date() for x in stock_holidays]

def check_tradingdayhour(day):
    trading_day = day
    if trading_day.hour >= 16:
        trading_day += timedelta(1)
    #Check if news article published on weekend. If so return monday. If day on holiday return next trading day
    while trading_day.weekday() in holidays.WEEKEND or trading_day in stock_holidays:
        trading_day += timedelta(1)
    return trading_day

output_file_path = "gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/"
output_file_name = "data_whole_woScAR_up.csv"

data_text = pd.read_csv("gdrive/My Drive/Thesis/processed data/processdata_woSWandPS_n.csv")

#CSV file from get_paneldata notebook
df = pd.read_csv("gdrive/My Drive/Thesis/processed data/CAR_regression/datasets_final/data_whole_woScAR.csv")


#Get both dfs to same date format. Later used for merging

data_text["Date"] = pd.to_datetime(data_text["Date"])
data_text["Date"] = [check_tradingdayhour(x) for x in data_text["Date"]]
data_text["Date"] = [x.strftime("%Y-%m-%d") for x in data_text["Date"]]

df["Date"] = pd.to_datetime(df["Date"])
df["Date"] = [check_tradingdayhour(x) for x in df["Date"]]
df["Date"] = [x.strftime("%Y-%m-%d") for x in df["Date"]]

#Merge rows with multiple news article issue of the same company on a given day. 
#This can be done either for the processed text or unprocessed text. Adjust variable in concatenate_text function
new_df = data_text.groupby(["Date", "Ticker"]).apply(concatenate_text).to_frame(name = "c_Text")
df = df.merge(new_df, how= "left", left_on = ["Date", "Ticker"], right_on = ["Date", "Ticker"])

df["c_Text"] = df["c_Text"].fillna(" ")

df["word_count"] = [count_words(x) for x in df.c_Text]

df.drop_duplicates(inplace = True)
df = df.rename(columns={'c_Text': 'Text_unprocessed'})
df_columns = ["Date", "Ticker", "Nasdaq", "Turnover", "Size", "BTM","pref_alpha", "Text_unprocessed", "word_count", "AR", "Return"] 
df = df[df_columns]
print(df)

with open(output_file_path + output_file_name, "w") as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(
        ["Date", "Ticker", "Nasdaq", "Turnover", "Size", "BTM",
         "pref_alpha", "Text_unprocessed", "word_count", "AR", "Return"])
    for index, row in df.iterrows():
        writer.writerow(row)






Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
              Date Ticker  Nasdaq  Turnover          Size       BTM  \
0       2015-01-02   AAPL       1  1.336802  6.370024e+08  0.187370   
1       2015-01-05   AAPL       1  1.344416  6.190077e+08  0.192817   
2       2015-01-06   AAPL       1  1.347419  6.190077e+08  0.192817   
3       2015-01-07   AAPL       1  1.345351  6.190660e+08  0.192799   
4       2015-01-08   AAPL       1  1.351682  6.277467e+08  0.190132   
...            ...    ...     ...       ...           ...       ...   
127053  2019-12-24    PEP       1  0.496903  1.910934e+08  0.077376   
127054  2019-12-26    PEP       1  0.498080  1.906053e+08  0.077574   
127055  2019-12-27    PEP       1  0.494766  1.906053e+08  0.077574   
127056  2019-12-30    PEP       1  0.497787  1.903027e+08  0.077697   
127057  2019-12-31    PEP       1  0.495126  1.903027e+08  0.077697   

        pref_alp